In [13]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
from sklearn.preprocessing import MinMaxScaler

class SOM:
    def __init__(self, grid_size=(3, 3), n_features=2, n_epochs=100, learning_rate=0.1, radius=2):
        self.grid_size = grid_size
        self.n_features = n_features
        self.n_epochs = n_epochs
        self.learning_rate = learning_rate
        self.radius = radius
        
        # Initialize the weight matrix with random values
        self.weights = np.random.rand(self.grid_size[0], self.grid_size[1], self.n_features)
        self.frames = []  # Store frames for GIF
        
    def _euclidean_distance(self, a, b):
        """Compute the Euclidean distance between two vectors."""
        return np.linalg.norm(a - b)
    
    def _find_bmu(self, data_point):
        """Find the Best Matching Unit (BMU) for a given data point."""
        min_dist = np.inf
        bmu_idx = (0, 0)
        for i in range(self.grid_size[0]):
            for j in range(self.grid_size[1]):
                dist = self._euclidean_distance(data_point, self.weights[i, j])
                if dist < min_dist:
                    min_dist = dist
                    bmu_idx = (i, j)
        return bmu_idx
    
    def _update_weights(self, data_point, bmu_idx, epoch):
        """Update the weights of the BMU and its neighbors."""
        # Reduce the learning rate and radius over time
        learning_rate = self.learning_rate * (1 - epoch / self.n_epochs)
        radius = self.radius * (1 - epoch / self.n_epochs)
        
        for i in range(self.grid_size[0]):
            for j in range(self.grid_size[1]):
                # Compute the distance from the BMU
                dist = np.linalg.norm(np.array([i, j]) - np.array(bmu_idx))
                
                # If the distance is within the neighborhood radius
                if dist <= radius:
                    influence = np.exp(-dist ** 2 / (2 * (radius ** 2)))
                    # Update the weights
                    self.weights[i, j] += learning_rate * influence * (data_point - self.weights[i, j])
    
    def fit(self, X, save_path="SOM_with_Grid_Overlay.gif"):
        # Normalize the data to [0, 1]
        scaler = MinMaxScaler()
        X = scaler.fit_transform(X)
        
        for epoch in range(self.n_epochs):
            np.random.shuffle(X)
            for data_point in X:
                # Find the BMU (Best Matching Unit)
                bmu_idx = self._find_bmu(data_point)
                # Update the weights of the BMU and its neighbors
                self._update_weights(data_point, bmu_idx, epoch)
            
            # Save the current state of the SOM with grid overlay
            self._save_plot(X, epoch)
        
        # Save GIF animation
        imageio.mimsave(save_path, self.frames, duration=0.3)
        print(f"GIF saved successfully as {save_path}")
    
    def _save_plot(self, X, epoch):
        """Visualizes the SOM grid overlaid on the data points over training epochs."""
        fig, ax = plt.subplots(figsize=(8, 8))
        
        # Plot the data points
        ax.scatter(X[:, 0], X[:, 1], c='gray', alpha=0.5, label='Data Points', edgecolors='k')
        
        # Plot the SOM grid (neurons)
        ax.scatter(self.weights[:, :, 0], self.weights[:, :, 1], c='red', s=100, marker='X', label='SOM Neurons')
        
        # Plot the grid structure (lines between neighboring neurons)
        for i in range(self.grid_size[0]):
            for j in range(self.grid_size[1]):
                if i + 1 < self.grid_size[0]:
                    ax.plot([self.weights[i, j, 0], self.weights[i + 1, j, 0]],
                            [self.weights[i, j, 1], self.weights[i + 1, j, 1]], 'r-', lw=1)
                if j + 1 < self.grid_size[1]:
                    ax.plot([self.weights[i, j, 0], self.weights[i, j + 1, 0]],
                            [self.weights[i, j, 1], self.weights[i, j + 1, 1]], 'r-', lw=1)

        # Title and labels
        ax.set_title(f'Self-Organizing Map (Epoch {epoch+1}/{self.n_epochs})', fontsize=14)
        ax.set_xlabel('Feature 1', fontsize=12)
        ax.set_ylabel('Feature 2', fontsize=12)
        ax.legend(loc='best', fontsize=12)
        
        ax.grid(True, linestyle='--', alpha=0.3)
        
        # Save the frame for GIF
        fig.canvas.draw()
        image = np.array(fig.canvas.renderer.buffer_rgba())
        self.frames.append(image)
        plt.close(fig)


# Generate synthetic data (random data points)
np.random.seed(42)
X = np.concatenate([
    np.random.randn(100, 2) * 0.5 + [1, 1],
    np.random.randn(100, 2) * 0.5 + [-1, -1],
    np.random.randn(100, 2) * 0.5 + [1, -1]
])

# Train SOM model and save GIF
model = SOM(grid_size=(3, 3), n_features=2, n_epochs=100, learning_rate=0.1, radius=2)
model.fit(X, save_path="SOM_with_Grid_Overlay.gif")


GIF saved successfully as SOM_with_Grid_Overlay.gif
